In [4]:
import os

In [ ]:
%pip install --upgrade accelerate
%pip uninstall -y transformers accelerate
%pip install transformers accelerate

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [5]:
%pwd

'e:\\TextSummarization\\research'

In [6]:
os.chdir("../")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class modeltrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int
    per_device_eval_batch_size: int

In [8]:
from textSummarizer.utils.common import read_yaml, create_directories
from textSummarizer.constants import *

In [9]:
class ConfigurationManager:
    
    def __init__(
        self,
        config_path = CONFIG_FILE_PATH,
        params_path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> modeltrainerConfig:
        
        config = self.config.model_trainer
        params = self.params.TrainingArguments
        
        create_directories([config.root_dir])
        
        model_trainer_config = modeltrainerConfig(
            root_dir = config.root_dir,
            data_path=config.data_path,
            num_train_epochs=params.num_train_epochs,
            weight_decay = params.weight_decay,
            model_ckpt=config.model_ckpt,
            evaluation_strategy=params.evaluation_strategy,
            logging_steps = params.logging_steps,
            warmup_steps=params.warmup_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,
            per_device_eval_batch_size=params.per_device_eval_batch_size
        )
        
        return model_trainer_config

In [10]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

c:\Users\hp\.conda\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
class modelTrainer:
    def __init__(self, config: modeltrainerConfig):
        self.config = config
        
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=pegasus)
        
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_eval_batch_size,
            weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )
        
        trainer = Trainer(model=pegasus, args=trainer_args,
                          tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                          train_dataset=dataset_samsum_pt["test"],
                          eval_dataset=dataset_samsum_pt["validation"]
        )
        
        trainer.train()
        
        
        pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

In [12]:
config = ConfigurationManager()
modeltrainer_config = config.get_model_trainer_config()
modeltrainer = modelTrainer(config=modeltrainer_config)
modeltrainer.train()

[2023-07-09 16:14:56,517: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-09 16:14:56,528: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-09 16:14:56,532: INFO: common: directory: artifacts created successfully]
[2023-07-09 16:14:56,535: INFO: common: directory: artifacts/model_trainer created successfully]


c:\Users\hp\.conda\envs\textS\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/51 [00:00<?, ?it/s]You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
